In [1]:
import os, sys, gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pandas import Series
from keras.layers import Conv1D, Dense, MaxPooling1D, Flatten
from keras.models import Sequential

%matplotlib inline

Using TensorFlow backend.


In [2]:
file_path = '/root/data/btc_data.pkl'
df = pd.read_pickle(file_path)
df.tail()

,Open,Close,Volume_(BTC),Percentage_Change,Expected_Return,BTC_Balance,USD_Balance,Total_Balance,Return,total_rewards
Timestamp,,,,,,,,,,
2017-05-30 23:56:00+00:00,2190.49,2181.37,1.700166,-0.000957,-9.12,0,0,0,0,0
2017-05-30 23:57:00+00:00,2190.50,2195.63,6.561029,0.006537,5.13,0,0,0,0,0
2017-05-30 23:58:00+00:00,2195.62,2191.83,15.662847,-0.001731,-3.79,0,0,0,0,0
2017-05-30 23:59:00+00:00,2195.82,2203.51,27.090309,0.005329,7.69,0,0,0,0,0
2017-05-31 00:00:00+00:00,2201.70,2208.33,9.961835,0.002187,6.63,0,0,0,0,0


In [3]:
initial_bal = 1000
num_steps = 1000000

In [4]:
# model's hyperparameters
window_size = 60
nb_filter = 10
filter_length = 10
nb_input_series = 10
nb_outputs = 2
epochs = 1
batch_size = 1

In [5]:
def environment(sate, action, fee): 

    action = actions[0] + actions[1]
    
    if action * state[3] > fee * state[0]: 
        state[2] = state[2] + (action * state[3] / state[0]  - fee)
        state[3] = state[3] - action * state[3] - fee * state[0]
        state[4] = state[3] + state[2] * state[0]
        return state

    elif (action * state[2] * -1) > fee:
        state[2] = state[2] - (action * state[2])
        state[3] = state[3] + (action * state[2] - fee * state[0])
        state[4] = state[3] + state[2] * state[0]
        return state
    else:
        return state

In [11]:
def loss(previous_state_value, current_state_value):
    loss = -1 * (current_state_value - previous_state_value) / (current_state_value + previous_state_value)
    return loss

In [6]:
def build_convolution1D_model(window_size, filter_length, nb_input_series=1, nb_outputs=1, nb_filter=4):
    model = Sequential((
        Conv1D(filters=nb_filter, kernel_size=filter_length, activation='relu', input_shape=(window_size, nb_input_series)),
        MaxPooling1D(), # Downsample the output of convolution by 2X.
        Conv1D(filters=nb_filter, kernel_size=filter_length, activation='relu'),
        MaxPooling1D(),
        Flatten(),
        Dense(nb_outputs, activation='linear'), # For binary classification, change the activation to 'sigmoid'
        ))
    model.compile(loss=loss, optimizer='adam', metrics=[loss]) # this part needs correction, instead of loss function the value of loss function should be passed, or maybe not, i need to double check this.
    return model

In [7]:
step = 0
Y_train = np.zeros((1, 2))
X_train = df[step:step + window_size].as_matrix()
X_train = np.reshape(X_train, (1, 60, 10))
print (X_train.shape)

(1, 60, 10)


In [12]:
# build the model and print the summary of the model
model = build_convolution1D_model(window_size=window_size, filter_length=filter_length, nb_input_series=nb_input_series, nb_outputs=nb_outputs, nb_filter=nb_filter)
print('\n\nModel with input size {}, output size {}, {} conv filters of length {}'.format(model.input_shape, model.output_shape, nb_filter, filter_length))
model.summary()



Model with input size (None, 60, 10), output size (None, 2), 10 conv filters of length 10
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 51, 10)            1010      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 25, 10)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 16, 10)            1010      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 8, 10)             0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 80)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 162       
Total params: 2,182
Trainable params: 2,182
Non-tr

In [13]:
model.fit(x=X_train, y=Y_train, batch_size=60, epochs=1, verbose=2, validation_data=None, shuffle=False)

Epoch 1/1
1s - loss_1: -1.0000e+00 - loss_2: -1.0000e+00
